In [1]:
%reload_ext autoreload
%autoreload 2
	
import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    # "/data/home/tranx/conda/envs/aligner_20240822_v2/python-packages", #CONDA_PREFIX/python-packages
    "/data/home/kapilk/.conda/envs/aligner_20240822_v2/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
import json
from pprint import pprint

import torch

In [20]:
# from llm_mm_aligner.evaluate import main as llm_mm_aligner_evaluate

# Dependency

In [152]:
# dependency
from transformers.trainer import TRAINING_ARGS_NAME
from typing import Any, Dict, List, Optional, Sequence, Union, cast

# fbcode/fblearner/flow/projects/assistant/multimodal/llm_mm_aligner/fblearner_utils.py
from itertools import chain
from dataclasses import dataclass

@dataclass
class JobRunnerArgs:
    trainer_args: Optional[Dict[str, Any]] = None
    eval_args: Optional[Dict[str, Any]] = None
    fsdp_config: Optional[Dict[str, Any]] = None
    log_dir: Optional[str] = None
    is_mast: Optional[bool] = False
    eval_output_path: Optional[str] = None

    def get_args_list(self, args: Dict[str, Any]) -> Sequence[str]:
        """Convert a dict of args to a list of strings for passing to the binary"""
        # TODO: This may introduce some confusion when v is set to false
        return list(
            chain.from_iterable(
                [f"--{k}", str(v)] if v is not None else [f"--{k}"]
                for k, v in args.items()
            )
        )

# fbcode/fblearner/flow/projects/assistant/multimodal/llm_mm_aligner/fblearner.py
import os
import torch

from transformers.trainer import TRAINING_ARGS_NAME
from llm_mm_aligner.lib.file_utils import get_pathmgr
from llm_mm_aligner.lib.configs import DataTrainingArguments

from llm_mm_aligner.lib.file_utils import (
    get_pathmgr,
    LM_EMBEDDING_NAME,
    LM_HEAD_NAME,
    PERCEPTION_NAME,
    PERCEPTION_TOKENIZER_NAME,
    TC_OUTPUT_FILE_PATH,
)

def manifold_file_exists(str):
    pass 

def extract_data_args(data_args: DataTrainingArguments) -> Dict[str, Any]:
    """
    Only keep the ones needed to align the image preprocessing setting with the training for offline evaluation.
    """
    processing_related_args = [
        "resize_longest",
        "add_resized_image",
        "chunk_size",
        "num_image_chunks",
        "activated_num_image_chunks",
        "max_total_image_chunks",
        "instruction_model_type",
        "frames_per_clip",
        "clips_per_video",
        "frame_dilation",
        "treat_frames_as_images",
    ]

    key_data_args = {k: getattr(data_args, k) for k in processing_related_args}
    return key_data_args


def is_in_sanitized_list(key):
    for keyword in [
        "fsdp",
        "accelerator_config",
        "lr_scheduler_kwargs",
    ]:
        if keyword in key:
            print("SANITIZE: ", key)
            return True
    exact_match = ["debug"]
    if key in exact_match:
        return True
    return False


def sanitize_eval_args_old(eval_args: Dict[str, Any]) -> Dict[str, Any]:
    """Removes any arguments passed from training/model args which are not
    needed for evaluation.
    """
    eval_args = {
        k: v
        for k, v in eval_args.items()
        if v is not None and not is_in_sanitized_list(k)
    }
    return eval_args

def sanitize_eval_args(eval_args: Dict[str, Any]) -> Dict[str, Any]:
    """Removes any arguments passed from training/model args which are not
    needed for evaluation.
    SANITIZE:  lr_scheduler_kwargs
SANITIZE:  fsdp
SANITIZE:  fsdp_min_num_params
SANITIZE:  fsdp_config
SANITIZE:  accelerator_config
SANITIZE:  using_fsdp2

    """
    sanitized_keys = ["debug", "fsdp", "fsdp_min_num_params", "fsdp_config", "accelerator_config"]
    
    eval_args = {
        k: v
        for k, v in eval_args.items()
        if v is not None and k not in sanitized_keys
        # if v is not None and not is_in_sanitized_list(k)
    }
    
    return eval_args


def load_and_merge_trainer_to_eval_args(
    job_runner_args: JobRunnerArgs,
) -> JobRunnerArgs:

    ckpt_path_to_load = None
    # must specify the "resume_from_checkpoint" in eval_args
    # pyre-fixme[16]: `Optional` has no attribute `get`.
    if job_runner_args.eval_args.get("resume_from_checkpoint", None) is not None:
        # pyre-fixme[16]: `Optional` has no attribute `__getitem__`.
        ckpt_path_to_load = job_runner_args.eval_args["resume_from_checkpoint"]

    if ckpt_path_to_load is None:
        raise ValueError("No resume_from_checkpoint found in eval_args.")

    # load trainer args from the ckpt path
    print(f"Running test flow on model ckpt {ckpt_path_to_load}")
    training_args_file = os.path.join(ckpt_path_to_load, TRAINING_ARGS_NAME)
    print(f"Reading training arguments from file {training_args_file}")
    if training_args_file.startswith("manifold://") and not manifold_file_exists(
        training_args_file
    ):
        raise ValueError(f"Training args file {training_args_file} does not exist.")

    # Copy training and model args to eval args. We only keep eval's copy of data and generation args
    with get_pathmgr().open(
        training_args_file,
        "rb",
    ) as fd:
        training_args = torch.load(fd, weights_only=False)
        model_args = training_args.model_args
        if not hasattr(training_args, "lr_scheduler_kwargs"):
            training_args.lr_scheduler_kwargs = {}
        trainer_training_args = training_args.to_dict()
        extracted_data_args = extract_data_args(training_args.data_args)
        trainer_model_args = model_args.to_dict()
        eval_args = job_runner_args.eval_args
        # There should be no overlap between training args and model args
        assert len(trainer_training_args.keys() & trainer_model_args) == 0
        # print("DEBUG trainer_training_args['using_fsdp2']:", trainer_training_args['using_fsdp2'])
        # print("DEBUG eval_args['using_fsdp2']:", eval_args['using_fsdp2'])
        # print("DEBUG is_in_sanitized_list('using_fsdp2'):",is_in_sanitized_list('using_fsdp2') )
        new_eval_args = {
            **trainer_training_args,
            **extracted_data_args,
            **trainer_model_args,
            **eval_args,
        }
        
        # print("DEBUG new_eval_args['using_fsdp2']:", new_eval_args['using_fsdp2'])
        
        print(f"eval args: {new_eval_args}")
        # Some eval args needs to be manually overriden/sanitized
        new_eval_args = sanitize_eval_args(new_eval_args)
        # new_eval_args = sanitize_eval_args_old(new_eval_args)
        # print("DEBUG 2 new_eval_args['using_fsdp2']:", new_eval_args['using_fsdp2'])
        # set resume path for the classifiers if needed
        if model_args.integrity_classifier or model_args.routing_classifier:
            # the model_name_or_path must be the resume path
            new_eval_args["model_name_or_path"] = new_eval_args[
                "resume_from_checkpoint"
            ]
        # set perception based on the freeze_perception arg
        
        # print("DEBUG 3 new_eval_args['using_fsdp2']:", new_eval_args['using_fsdp2'])
        if not model_args.freeze_perception:
            perception_path = os.path.join(ckpt_path_to_load, PERCEPTION_NAME)
            print(f"Setting perception to {perception_path}")
            new_eval_args["checkpoints_perception"] = perception_path
        # set perception tokenizer
        perception_tokenizer_path = os.path.join(
            ckpt_path_to_load, PERCEPTION_TOKENIZER_NAME
        )
        # in the case, where we use frozen perception tokenizer
        # we assume "checkpoints_perception_tokenizer" is already set
        if not model_args.freeze_tokenizer:
            new_eval_args["checkpoints_perception_tokenizer"] = (
                perception_tokenizer_path
            )
        print(
            f"Setting perception tokenizer path to {new_eval_args['checkpoints_perception_tokenizer']}"
        )
        if not model_args.freeze_lm:
            new_eval_args["checkpoints_lm"] = ckpt_path_to_load
            print(f"Setting lm checkpoint path to {new_eval_args['checkpoints_lm']}")
        # Set LORA if needed
        if ("lora" in new_eval_args) and new_eval_args["lora"]:
            new_eval_args["lora_checkpoint"] = ckpt_path_to_load
            if model_args.integrity_classifier:
                new_eval_args["lora_checkpoint"] = os.path.join(
                    ckpt_path_to_load, "adapter"
                )
            print(
                f"Setting LORA checkpoint path to {new_eval_args['lora_checkpoint'] }"
            )

        if ("lora_tokenizer" in new_eval_args) and new_eval_args["lora_tokenizer"]:
            new_eval_args["lora_tokenizer_checkpoint"] = os.path.join(
                ckpt_path_to_load, "adapter_tokenizer"
            )
            print(
                f"Setting LORA tokenizer checkpoint path to {new_eval_args['lora_tokenizer_checkpoint'] }"
            )

        if ("lora_perception" in new_eval_args) and new_eval_args["lora_perception"]:
            new_eval_args["lora_perception_checkpoint"] = os.path.join(
                ckpt_path_to_load, "adapter_perception"
            )
            print(
                f"Setting LORA perception checkpoint path to {new_eval_args['lora_perception_checkpoint'] }"
            )
        if model_args.add_grounding_tokens:
            # load embedding & head
            if model_args.use_metaformers:
                lm_embedding_path = ckpt_path_to_load
                lm_head_path = ckpt_path_to_load
            else:
                lm_embedding_path = os.path.join(ckpt_path_to_load, LM_EMBEDDING_NAME)
                lm_head_path = os.path.join(ckpt_path_to_load, LM_HEAD_NAME)
            print(f"Setting checkpoints_lm_embedding to {lm_embedding_path}")
            print(f"Setting checkpoints_lm_head to {lm_head_path}")
            new_eval_args["checkpoints_lm_embedding"] = lm_embedding_path
            new_eval_args["checkpoints_lm_head"] = (
                lm_head_path if model_args.add_grounding_llm_head_if_available else None
            )
        print(f"eval args after sanitization: {new_eval_args}")
        # Set the new eval args in job runner
        job_runner_args.eval_args = new_eval_args
    return job_runner_args

# Read training_args

In [7]:
# checkpoint_path = "/fsx_0/checkpoints/mm10/MM10-Stage1-70B/MH21_70B_224px_norm_R2/checkpoint-1000"
# path = f"/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/checkpoint-1000/training_args.bin"
checkpoint_path = "/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp32a_lr1_n128/checkpoint-8000"
with open(f"{checkpoint_path}/training_args.bin", "rb") as fd:
    training_args = torch.load(fd, weights_only=False)
    
model_args = training_args.model_args
model_args = model_args.to_dict()
training_args = training_args.to_dict()

In [8]:
for k, v in model_args.items():
    print(k, ":", v)

model_name_or_path : /fsx_0/checkpoints/llama3/mh21
tokenizer_path : /fsx_0/checkpoints/llama3/mh21
use_metaformers : True
recompute_attn : True
recompute_fc1_fc3 : True
pretraining_tp : 1
model_parallel_size : 8
context_parallel_size : 1
pipeline_parallel_size : 1
virtual_pipeline_parallel_size : 1
use_alternate_pipeline_parallel_config : True
less_layer_first_pp_stage : 0
less_layer_last_pp_stage : 0
use_te : True
parallelize_llm : True
parallelize_perception_tokenizer : True
use_sdpa : True
use_fused_layernorm : True
max_parallel_model_loading : 8
model_type : None
lm_mh_tokenizer_version : tiktoken_v5
ensemble_tokenizer_params : {}
use_ensemble_modality_tokenizer : False
modality_tokenizer_name : /fsx_0/user/zhenq/checkpoints/huggingface/ks336_29_38907_I336B160E327m_e3
use_fast_tokenizer : True
n_prefix_embs : 65
num_classes : None
routing_classifier : False
integrity_classifier : False
freeze_perception : False
freeze_perception_except_pos_emb : False
ignore_mismatched_sizes : Fal

In [3]:
training_args.keys()

dict_keys(['output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_backend', 'bf16_full_eval', 'fp16_

In [31]:
training_args['using_fsdp2']

True

# AWS Input config

In [113]:
eval_config_file = "/fsx_0/user/tranx/experiments/eval/sft_eval_overwrite_aws/mmmu.json"
with open(eval_config_file, 'r') as f:
    args = json.load(f)
    
print(args.keys())

OUTPUT_DIR="/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a"
CHECKPOINT_ID = 1000
BENCHMARK = "mmmu"
args["eval_args"]["resume_from_checkpoint"] = f"{OUTPUT_DIR}/checkpoint-{CHECKPOINT_ID}"
args["eval_args"]["tb_logdir"] = f"{OUTPUT_DIR}/tensorboard"
pprint(args['eval_args'])

OUTPUT_RESULT = f"{OUTPUT_DIR}/evals/eval_results_checkpoint-{CHECKPOINT_ID}/{BENCHMARK}_eval_results.json"

dict_keys(['scheduler_type', 'eval_only', 'num_gpus', 'num_nodes', 'eval_args'])
{'add_bos_token': False,
 'add_eos_token': False,
 'batch_size_generation': 2,
 'dataloader_num_workers': 4,
 'eval_ckpt': 2520,
 'eval_only': True,
 'eval_type': 'eval_mmmu',
 'generation_task': 'MMMUGenerationTask',
 'instr_prompt': 'mmmu',
 'max_length': 50,
 'max_new_tokens': 1024,
 'max_seq_len': 1024,
 'model_parallel_size': 8,
 'num_beams': 1,
 'perception_tokenizer_attention_dropout_p': 0,
 'perception_tokenizer_hidden_dropout_p': 0,
 'repetition_penalty': 1.05,
 'resume_from_checkpoint': '/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/checkpoint-1000',
 'stopping_token_ids': '',
 'task_type': 'instruction_tune',
 'tb_logdir': '/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/tensorboard',
 'train_file': '/fsx_0/dataset01/MMMU/mmmu_validation_v3.jsonl',
 'validation_file': '/fsx_0/dataset01/MMMU/mmmu_validation_v3.jsonl'}


In [34]:
# eval_config_file = "/fsx_0/user/tranx/experiments/eval/sft_eval_overwrite_aws/mmmu.json"
eval_config_file = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10/stage1/eval_overwrite/eval_mmmu.json"
with open(eval_config_file, 'r') as f:
    args = json.load(f)
    
OUTPUT_RESULT = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/evals/eval_results_checkpoint-1000/mmmu_eval_results.json"

In [33]:
model_args, data_args, generation_args, training_args = create_eval_job_args(args)

NameError: name 'create_eval_job_args' is not defined

In [136]:
generation_args.generation_output_dir

'/tmp/evaluation'

# AWS Shim Evaluate

In [114]:
job_runner_args = JobRunnerArgs(
    trainer_args=None,
    eval_args=args["eval_args"],
    fsdp_config={},
    log_dir="",
)

# must load trainer args from eval_args.resume_from_checkpoint
job_runner_args = load_and_merge_trainer_to_eval_args(job_runner_args)

# assign eval result output file    
job_runner_args.eval_output_path = OUTPUT_RESULT
pprint(job_runner_args)

Running test flow on model ckpt /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/checkpoint-1000
Reading training arguments from file /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/checkpoint-1000/training_args.bin
eval args: {'output_dir': '/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 48, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 1, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 5e-06, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 3.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 0, 'log_level': 'info', '

In [85]:
from transformers import HfArgumentParser

from llm_mm_aligner.lib.configs import (
    DataTrainingArguments,
    GenerationArguments,
    ModelArguments,
    TrainingArguments,
)

args = job_runner_args 

parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, GenerationArguments, TrainingArguments)
)
(
    model_args,
    data_args,
    generation_args,
    training_args,
) = parser.parse_args_into_dataclasses(
    args=args.get_args_list(args.eval_args),
)

model_args = cast(ModelArguments, model_args)
data_args = cast(DataTrainingArguments, data_args)
generation_args = cast(GenerationArguments, generation_args)
training_args = cast(TrainingArguments, training_args)


/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_20240822_v2/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_20240822_v2/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_20240822_v2/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


In [121]:
args["fsdp_config"]

KeyError: 'fsdp_config'

In [112]:
pprint(data_args)

DataTrainingArguments(dataset_name='MMdataset',
                      eval_dataset_name=None,
                      train_file='/fsx_0/dataset01/MMMU/mmmu_validation_v3.jsonl',
                      mixin_file=None,
                      validation_file='/fsx_0/dataset01/MMMU/mmmu_validation_v3.jsonl',
                      filetype='jsonlines',
                      mixin_filetype='jsonlines',
                      datarecipe_json_str=None,
                      use_hive_dataset=False,
                      use_mdl=False,
                      hive_batch_size=64,
                      hive_table_name=None,
                      max_train_samples=None,
                      max_seq_len=1024,
                      max_tokens_in_batch_row=None,
                      max_tokens_in_batch=None,
                      max_eval_samples=None,
                      validation_split_percentage=5,
                      instructions='',
                      instruction_model_type='MetaAiTikTokv4Ch

In [105]:
training_args.fsdp_config

{'min_num_params': 0,
 'xla': False,
 'xla_fsdp_v2': False,
 'xla_fsdp_grad_ckpt': False}

In [103]:
training_args["fsdp_config"]

TypeError: 'TrainingArguments' object is not subscriptable

In [128]:
pprint(training_args)

TrainingArguments(output_dir='/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a',
                  overwrite_output_dir=True,
                  do_train=True,
                  do_eval=False,
                  do_predict=False,
                  eval_strategy=<IntervalStrategy.NO: 'no'>,
                  prediction_loss_only=False,
                  per_device_train_batch_size=8,
                  per_device_eval_batch_size=48,
                  per_gpu_train_batch_size=None,
                  per_gpu_eval_batch_size=None,
                  gradient_accumulation_steps=1,
                  eval_accumulation_steps=None,
                  eval_delay=0.0,
                  learning_rate=5e-06,
                  weight_decay=0.1,
                  adam_beta1=0.9,
                  adam_beta2=0.95,
                  adam_epsilon=1e-08,
                  max_grad_norm=1.0,
                  num_train_epochs=3.0,
                  max_steps=-1,
                  lr_scheduler

In [129]:
training_args.output_dir

'/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a'

In [134]:
generation_args.generation_output_dir = None

In [95]:
args.eval_output_path

'/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/evals/eval_results_checkpoint-1000/mmmu_eval_results.json'

In [96]:
from llm_mm_aligner.evaluate import main as llm_mm_aligner_evaluate

eval_metric = llm_mm_aligner_evaluate(
    model_args,
    data_args,
    training_args,
    generation_args,
    args.eval_output_path,
)

fb dependencies not available, skipping init_aix


AssertionError: 

# Overwrite both eval and training config

In [162]:
eval_config_file = "/fsx_0/checkpoints/mm10/MM10-Stage1-70B/MH21_70B_224px_norm_R2/evals/eval_results_checkpoint-2800/mmmu_2800_eval_config.json"
with open(eval_config_file, 'r') as f:
    input_arguments = json.load(f)
    
    
input_arguments["eval_args"].update({
    "instruction_model_type": "MetaAiTikTokv4ChatFormat",
    "lm_mh_tokenizer_version": "meta_ai_tiktoken_v4"
    
})
# input_arguments["eval_args"]["using_fsdp2"] = False
# input_arguments['eval_args']

In [149]:
input_arguments["eval_args"]["instruction_model_type"]

KeyError: 'instruction_model_type'

In [ ]:
# v4
"instruction_model_type": "MetaAiTikTokv4ChatFormat"
lm_mh_tokenizer_version: Optional[str] = field( 
    default="meta_ai_tiktoken_v4",
    metadata={
        "help": "MH LM tokenizer version like meta_ai_tiktoken_v4 or tiktoken_v5"
    },
)
# v5
"instruction_model_type": "TikTokv5ChatFormat",
"lm_mh_tokenizer_version": "tiktoken_v5",

In [163]:
OUTPUT_RESULT = "/tmp"

args = JobRunnerArgs(
    trainer_args=None,
    eval_args=input_arguments["eval_args"],
    fsdp_config={},
    log_dir="",
)

args = load_and_merge_trainer_to_eval_args(args)

Running test flow on model ckpt /fsx_0/checkpoints/mm10/MM10-Stage1-70B/MH21_70B_224px_norm_R2/checkpoint-2800
Reading training arguments from file /fsx_0/checkpoints/mm10/MM10-Stage1-70B/MH21_70B_224px_norm_R2/checkpoint-2800/training_args.bin
eval args: {'output_dir': '/fsx_0/checkpoints/mm10/MM10-Stage1-70B/MH21_70B_224px_norm_R2/tensorboard', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 48, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 4, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 0.0001, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 100.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 200, 'log_level': 'in

In [164]:
print(args.trainer_args, args.eval_args['instruction_model_type'])

None MetaAiTikTokv4ChatFormat


In [165]:
from transformers import HfArgumentParser
from llm_mm_aligner.lib.configs import (
    DataTrainingArguments,
    GenerationArguments,
    ModelArguments,
    TrainingArguments,
)

parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, GenerationArguments, TrainingArguments)
)
(
    model_args,
    data_args,
    generation_args,
    training_args,
) = parser.parse_args_into_dataclasses(
    args=args.get_args_list(args.eval_args),
)

/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aws/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aws/lib/python3.11/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


In [161]:
print(data_args.instruction_model_type, model_args.lm_mh_tokenizer_version)

TikTokv5ChatFormat tiktoken_v5


In [166]:
print(data_args.instruction_model_type, model_args.lm_mh_tokenizer_version)

MetaAiTikTokv4ChatFormat meta_ai_tiktoken_v4
